In [31]:
import cv2 as cv
import numpy as np
from functions import analyse
import os
from os.path import join

from datetime import datetime as dt
def frame_stamp():
    current = dt.now()
    out_str = (str(current.year) + str(current.month) + str(current.day) + "_" + 
               str(current.hour) + str(current.minute) + str(current.second) + "_" +
               "frame")
    return out_str



ssd = cv.dnn.readNetFromCaffe(join(os.getcwd(), 'resources', 'MobileNetSSD_deploy.prototxt'), 
                          join(os.getcwd(), 'resources', 'MobileNetSSD_deploy.caffemodel'))
classifier = cv.dnn.readNetFromTensorflow(join(os.getcwd(), 'frozen_model.pb'))
detector = cv.xfeatures2d.SURF_create(2000)
detector.setUpright(True)

move_frame = False
pts = []
def next_frame(event,x,y,flags,param):
    if event == cv.EVENT_LBUTTONDOWN:
        global pts
        pts = []
        pts.append(x)
        pts.append(y)

    elif event == cv.EVENT_LBUTTONUP:
        if (pts[0] < x) or (pts[1] < y):
            pts.append(x)
            pts.append(y)
            global move_frame
            move_frame = True
            
def classify(file = 'resources/test_video.mp4', width = 960, height = 540, flip = True):
    cap = cv.VideoCapture(file)
    ret, frame = cap.read()
    clone = cv.resize(frame, (width,height))
    if(flip): clone = cv.flip(clone, 0)
    cv.namedWindow("Video")
    cv.setMouseCallback("Video", next_frame)
    global move_frame
    global threshold
    threshold = 2000
    
    ## Loop to classify Images
    while ret:
        cv.imshow("Video", clone)
        key = cv.waitKey(1)
        
        skip = False
        if (key == 113):
            break
        if (key == 46):
            skip = True
        
        ## When image is classified or frame is skipped
        if (move_frame or skip):
            ret, frame = cap.read()
            if (skip):
                clone = cv.resize(frame, (width,height))
                if(flip): clone = cv.flip(clone, 0)
            elif (move_frame):
                move_frame = False
                roi = clone[pts[1]:pts[3], pts[0]:pts[2]]
                
                ## Some issues with the dector functiom not putting the images in the centre
                
#                 global detector
#                 keypoints, detector, threshold, startX, endX = analyse(frame, ssd, classifier, detector, 0,
#                                                             threshold, use_classifier=False)
#                 for i in range(len(keypoints)):
#                     x_img = int(keypoints[i].pt[0])
#                     y_img = int(keypoints[i].pt[1])
#                     cv.circle(frame, (x_img,y_img), 10, (0,255,0), 1)
#                 clone = cv.resize(frame, (width,height))    
#                 clone = cv.flip(clone, 0) 
#                 print("Keypoints: ", keypoints)
                
                cv.imshow("ROI", roi)
                cv.imwrite("./labels/" + frame_stamp() + ".jpg", roi)
                cv.waitKey(0)
                cv.destroyWindow("ROI")

    cap.release()
    cv.destroyAllWindows()
    
    
def play_video(file = 'resources/test_video.mp4', width = 960, height = 540, flip = True):
    cap = cv.VideoCapture(file)
    ret, frame = cap.read()
    clone = cv.resize(frame, (width,height))
    if(flip): clone = cv.flip(clone, 0)
    while ret:
        cv.imshow("Input", clone)
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
        ret, frame = cap.read()
        clone = cv.resize(frame, (width,height))
        if(flip): clone = cv.flip(clone, 0)
    cap.release()
    cv.destroyAllWindows()

In [32]:
classify()